# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [378]:
!pip install cache-magic

  Created wheel for cache-magic: filename=cache_magic-1.0.4-py3-none-any.whl size=6703 sha256=e9ef3193b25191edfea94b00e685ecc725cdf2ee8f874cabec792caafd04ab1c
  Stored in directory: c:\users\zhouc\appdata\local\pip\cache\wheels\24\16\f2\016dbddf24f703cf51ad82bb88ca3a77458b20199acd3a5590
Successfully built cache-magic


In [298]:
cat_count = (df.iloc[:, 4:] != 0).sum()

In [299]:
cat_count

related                   19906
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire                        282
earthqua

In [388]:
%cache cache1 = model_cv

Creating new value for variable 'cache1'


PicklingError: Can't pickle <function tokenize at 0x0000022E4B4A5CA8>: it's not the same object as __main__.tokenize

In [376]:
import dill
dill.dump_session('notebook_env.db')

TypeError: can't pickle dict_keys objects

In [ ]:
#change if needed
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sys
import cache_magic

#machine learning
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report, fbeta_score
from sklearn.svm import SVC
from scipy.stats.mstats import gmean

from sklearn.base import BaseEstimator, TransformerMixin

#NLP
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline, FeatureUnion
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

%matplotlib inline

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_response.db')
df = pd.read_sql_table(table_name='message_categories', con=engine)
X = df['message']
Y = df.iloc[:,4:]

In [7]:
df.isna().sum()

id                            0
message                       0
original                  15990
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_infrastructure          0
weather_

In [4]:
display(X, Y)

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26023    The training demonstrated how to enhance micro...
26024    A suitable candidate has been selected and OCH...
26025    Proshika, operating in Cox's Bazar municipalit...
26026    Some 2,000 women protesting against the conduc...
26027    A radical shift in thinking came about as a re...
Name: message, Length: 26028, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26024,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26025,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26026,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''
    IN: 
        raw text for tokenizing via the following steps: 
            - normalized, punctuation removed, stop words removed, stemmed, and lemmatized
    OUT:
        tokenized text
    '''
    #Normalize text and remove punctuation
    normalized_txt = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize text
    words = word_tokenize(normalized_txt)
 
    #lemmatize
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    #Reduce words to their stems
    words = [PorterStemmer().stem(w) for w in words]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
#starting pipeline
def pipeline_model():
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                          ('tfidf', TfidfTransformer()),
                          ('clf', MultiOutputClassifier(RandomForestClassifier()))])
    return pipeline

In [9]:
model = pipeline_model()

In [10]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,random_state = 42)

ValueError: Found input variables with inconsistent numbers of samples: [6, 26028]

In [12]:
%%time
model.fit(X_train, Y_train)

Wall time: 4min 57s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
%%time
#make predictions
y_preds = model.predict(X_test)

Wall time: 16.2 s


In [179]:
def metrics_report(Y_test, y_preds):
    '''
    Create a weighted averages summary dataframe for each label, 
    using the classification_report function 
    IN: 
        Y_test - array of actual values
        y_preds - numpy array of predicted values
    OUT df columns: precision,recall,f1-score,support
    This is a performance metric reference from Github User: rebeccaebarnes
    '''
    results_dict = {}

    for pred, label, col in zip(y_preds.transpose(), Y_test.values.transpose(), Y_test.columns):
        results_dict[col] = classification_report(label, pred, output_dict=True)
        
    weighted_avg = {}
    for key in results_dict.keys():
        weighted_avg[key] = results_dict[key]['weighted avg']

    df_wavg = pd.DataFrame(weighted_avg).transpose()
    return df_wavg

In [20]:
def weighted_avg_report(df):
    '''
    OUT:
        descriptive statistics for the created weighted averages summary df
        upper and lower quantile df slices
    '''
    display(df['f1-score'].describe())
    display('lowest quantile of f scores',df[df['f1-score'] <= df['f1-score'].quantile(0.25)]) # lowest quantile of f scores
    #print(df.sort_values('f1-score').head(n = 10))
    display('highest quantile of f scores', df[df['f1-score'] >= df['f1-score'].quantile(0.75)]) # highest quantile of f scores

In [169]:
weighted_avg_report(df_wavg)

count    35.000000
mean      0.931140
std       0.057876
min       0.771124
25%       0.916096
50%       0.944198
75%       0.971679
max       0.994010
Name: f1-score, dtype: float64

'lowest quantile of f scores'

,precision,recall,f1-score,support
related,0.798599,0.806209,0.771124,6507.0
request,0.893981,0.896266,0.882489,6507.0
aid_related,0.784650,0.783157,0.778731,6507.0
medical_help,0.902344,0.918549,0.883507,6507.0
food,0.922070,0.926387,0.914761,6507.0
other_aid,0.845730,0.873060,0.817647,6507.0
infrastructure_related,0.875334,0.935147,0.904252,6507.0
weather_related,0.875941,0.877363,0.872271,6507.0
direct_report,0.862917,0.861380,0.836987,6507.0


'highest quantile of f scores'

,precision,recall,f1-score,support
offer,0.992025,0.996004,0.994010,6507.0
clothing,0.983620,0.985708,0.980129,6507.0
missing_people,0.989809,0.989703,0.984731,6507.0
electricity,0.979456,0.981405,0.973295,6507.0
tools,0.987743,0.993853,0.990789,6507.0
hospitals,0.978296,0.989089,0.983663,6507.0
shops,0.989271,0.994621,0.991939,6507.0
aid_centers,0.977992,0.988935,0.983433,6507.0
fire,0.977991,0.988781,0.983356,6507.0


In [21]:
weighted_avg = {}
for key in results_dict.keys():
    weighted_avg[key] = results_dict[key]['weighted avg']

df_wavg = pd.DataFrame(weighted_avg).transpose()
df_wavg

,precision,recall,f1-score,support
related,0.798599,0.806209,0.771124,6507.0
request,0.893981,0.896266,0.882489,6507.0
offer,0.992025,0.996004,0.994010,6507.0
aid_related,0.784650,0.783157,0.778731,6507.0
medical_help,0.902344,0.918549,0.883507,6507.0
medical_products,0.946887,0.949900,0.929101,6507.0
search_and_rescue,0.972233,0.971415,0.958197,6507.0
security,0.966493,0.979407,0.969668,6507.0
military,0.951597,0.965576,0.951062,6507.0
water,0.946390,0.948210,0.933750,6507.0


In [22]:
df_wavg['f1-score'].describe()

count    35.000000
mean      0.931140
std       0.057876
min       0.771124
25%       0.916096
50%       0.944198
75%       0.971679
max       0.994010
Name: f1-score, dtype: float64

In [23]:
df_wavg[df_wavg['f1-score'] <= df_wavg['f1-score'].quantile(0.25)]

,precision,recall,f1-score,support
related,0.798599,0.806209,0.771124,6507.0
request,0.893981,0.896266,0.882489,6507.0
aid_related,0.784650,0.783157,0.778731,6507.0
medical_help,0.902344,0.918549,0.883507,6507.0
food,0.922070,0.926387,0.914761,6507.0
other_aid,0.845730,0.873060,0.817647,6507.0
infrastructure_related,0.875334,0.935147,0.904252,6507.0
weather_related,0.875941,0.877363,0.872271,6507.0
direct_report,0.862917,0.861380,0.836987,6507.0


In [25]:
df_wavg.sort_values('f1-score').head(n = 10)

,precision,recall,f1-score,support
related,0.798599,0.806209,0.771124,6507.0
aid_related,0.784650,0.783157,0.778731,6507.0
other_aid,0.845730,0.873060,0.817647,6507.0
direct_report,0.862917,0.861380,0.836987,6507.0
weather_related,0.875941,0.877363,0.872271,6507.0
request,0.893981,0.896266,0.882489,6507.0
medical_help,0.902344,0.918549,0.883507,6507.0
infrastructure_related,0.875334,0.935147,0.904252,6507.0
food,0.922070,0.926387,0.914761,6507.0
shelter,0.927412,0.932073,0.917431,6507.0


In [27]:
df_wavg[df_wavg['f1-score'] >= df_wavg['f1-score'].quantile(0.75)]

,precision,recall,f1-score,support
offer,0.992025,0.996004,0.994010,6507.0
clothing,0.983620,0.985708,0.980129,6507.0
missing_people,0.989809,0.989703,0.984731,6507.0
electricity,0.979456,0.981405,0.973295,6507.0
tools,0.987743,0.993853,0.990789,6507.0
hospitals,0.978296,0.989089,0.983663,6507.0
shops,0.989271,0.994621,0.991939,6507.0
aid_centers,0.977992,0.988935,0.983433,6507.0
fire,0.977991,0.988781,0.983356,6507.0


---


In [38]:
df_feats = df.iloc[:,4:]

In [39]:
df_feats_counts = df_feats.sum()

In [42]:
(df_feats_counts/df_feats.shape[0]).sort_values()

offer                     0.004534
shops                     0.004610
tools                     0.006109
fire                      0.010834
hospitals                 0.010873
missing_people            0.011449
aid_centers               0.011872
clothing                  0.015560
security                  0.018096
cold                      0.020363
electricity               0.020440
money                     0.023206
search_and_rescue         0.027816
military                  0.033041
refugees                  0.033618
other_infrastructure      0.044222
death                     0.045874
transport                 0.046143
medical_products          0.050446
buildings                 0.051214
other_weather             0.052866
water                     0.064239
infrastructure_related    0.065506
medical_help              0.080068
floods                    0.082795
shelter                   0.088904
storm                     0.093860
earthquake                0.094321
food                

In [43]:
df_wavg['f1-score'].sort_values()

related                   0.771124
aid_related               0.778731
other_aid                 0.817647
direct_report             0.836987
weather_related           0.872271
request                   0.882489
medical_help              0.883507
infrastructure_related    0.904252
food                      0.914761
shelter                   0.917431
other_weather             0.926304
buildings                 0.926983
medical_products          0.929101
storm                     0.929327
water                     0.933750
other_infrastructure      0.935546
transport                 0.940107
death                     0.944198
floods                    0.947253
military                  0.951062
refugees                  0.952750
search_and_rescue         0.958197
money                     0.963854
earthquake                0.967192
security                  0.969668
cold                      0.970064
electricity               0.973295
clothing                  0.980129
fire                

In [36]:
df.shape[0]

26028

In [ ]:
df_feats.sort_values()

---

### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000022E4B4A5CA8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [ ]:
parameters = {'vect__ngram_range':[(1,2),(2,2)],
            'clf__estimator__n_estimators':[50, 100]
             }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)

In [46]:
%%time
#grid search

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25, 50, 100], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

model_cv = GridSearchCV(estimator=model, param_grid=parameters, verbose=3)

Wall time: 0 ns


In [49]:
%%time
model_cv.fit(X_train, Y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.218, total=  35.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.3s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.225, total=  36.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.225, total=  36.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.239, total=  35.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.235, total=  38.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.226, total=  34.5s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__mi

[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 379.4min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [50]:
model_cv.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 100,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

In [51]:
model_cv.best_score_

0.2567491761298042

In [53]:
%%time
y_preds_cv = model_cv.predict(X_test)

Wall time: 16.1 s


In [55]:
cv_results = model_performance(Y_test, y_preds)

related
              precision    recall  f1-score   support

           0       0.76      0.29      0.42      1569
           1       0.81      0.97      0.88      4938

    accuracy                           0.81      6507
   macro avg       0.79      0.63      0.65      6507
weighted avg       0.80      0.81      0.77      6507

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5403
           1       0.87      0.46      0.60      1104

    accuracy                           0.90      6507
   macro avg       0.88      0.72      0.77      6507
weighted avg       0.89      0.90      0.88      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6374
           1       0.33      0.01      0.01       133

    accuracy                           0.98      6507
   macro avg       0.66      0.50      0.50      6507
weighted avg       0.97      0.98      0.97      6507

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6282
           1       0.53      0.04      0.07       225

    accuracy                           0.97      6507
   macro avg       0.75      0.52      0.53      6507
weighted avg       0.95      0.97      0.95      6507

water
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6077
           1       0.91      0.24      0.38       430

    accuracy                           0.95      6507
   macro avg       0.93      0.62      0.68      6507
weighted avg       0.95      0.95      0.93      6507

foo

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

---
Try new model parameters

In [90]:
%%time
#grid search

parameters_2 = {'vect__min_df': [5, 10],
              'tfidf__use_idf':[True],
              'vect__binary':[True, False],
              'clf__estimator__n_estimators':[100, 500, 1000], 
              'clf__estimator__min_samples_split':[1, 2]}

model_cv_2 = GridSearchCV(estimator=model, param_grid=parameters_2, verbose=3)

Wall time: 0 ns


In [10]:
#pipeline model 2
def pipeline_model_2():
    '''
    build a pipeline model with the specificed transformers and estimator
    '''
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            #('starting_verb', StartingVerbExtractor()),
            #("word_count", WordCount()),
            #("character_count", CharacterCount()),
            #("noun_count", NounCount()),
            #("verb_count", VerbCount())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    #pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
    #                      ('tfidf', TfidfTransformer()),
    #                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])
    return pipeline

In [50]:
avg_accuracy_model_2 = (y_preds_2 == Y_test).mean().mean()
avg_accuracy_model_2

0.9452633427737163

In [11]:
model_2 = pipeline_model_2()

In [79]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,random_state = 42)

In [13]:
%%time
model_2.fit(X_train, Y_train)

Wall time: 4min 54s


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [14]:
%%time
#make predictions
y_preds_2 = model_2.predict(X_test)

Wall time: 18.5 s


In [18]:
wavg_df_2 = create_report_summary(y_preds_2, Y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
weighted_avg_report(wavg_df_2)

count    35.000000
mean      0.931105
std       0.058078
min       0.768918
25%       0.921070
50%       0.942290
75%       0.971146
max       0.994010
Name: f1-score, dtype: float64

'lowest quantile of f scores'

,precision,recall,f1-score,support
related,0.794046,0.803903,0.768918,6507.0
request,0.896604,0.897341,0.883021,6507.0
aid_related,0.783032,0.781927,0.777688,6507.0
medical_help,0.904734,0.919164,0.885168,6507.0
shelter,0.929301,0.933303,0.919152,6507.0
other_aid,0.833315,0.872291,0.816404,6507.0
infrastructure_related,0.875325,0.934993,0.904176,6507.0
weather_related,0.874433,0.875980,0.870815,6507.0
direct_report,0.860175,0.860612,0.836897,6507.0


'highest quantile of f scores'

,precision,recall,f1-score,support
offer,0.992025,0.996004,0.994010,6507.0
clothing,0.982804,0.985400,0.979467,6507.0
missing_people,0.989809,0.989703,0.984731,6507.0
electricity,0.981603,0.981251,0.972675,6507.0
tools,0.987743,0.993853,0.990789,6507.0
hospitals,0.978296,0.989089,0.983663,6507.0
shops,0.989271,0.994621,0.991939,6507.0
aid_centers,0.977992,0.988935,0.983433,6507.0
fire,0.977992,0.988935,0.983433,6507.0


In [144]:
model_cv_2 = GridSearchCV(estimator=model, param_grid=parameters_2, verbose=3)

In [147]:
%%time
model_cv_2.fit(X_train, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__estimator__min_samples_split=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.1s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5, score=nan, total=  12.2s
[CV] clf__estimator__min_samples_split=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.8s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5, score=nan, total=  12.7s
[CV] clf__estimator__min_samples_split=1, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5 


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)


[CV]  clf__estimator__min_samples_split=1, clf__estimator__n_estimators=1000, tfidf__use_idf=True, vect__binary=False, vect__min_df=10, score=nan, total=  13.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5, score=0.260, total= 3.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5, score=0.257, total= 4.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, tfidf__use_idf=True, vect__binary=True, vect__

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 1480.1min finished


Wall time: 1d 1h 34min 2s


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [148]:
model_cv_2.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 1000,
 'tfidf__use_idf': True,
 'vect__binary': False,
 'vect__min_df': 10}

In [149]:
model_cv_2.best_score_

0.2649457268949014

In [150]:
%%time
y_preds_cv_2 = model_cv_2.predict(X_test)

Wall time: 4min 5s


In [151]:
%%time
cv_2_results = model_performance(Y_test, y_preds_cv_2)

related
              precision    recall  f1-score   support

           0       0.75      0.30      0.43      1569
           1       0.81      0.97      0.88      4938

    accuracy                           0.81      6507
   macro avg       0.78      0.64      0.66      6507
weighted avg       0.80      0.81      0.77      6507

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5403
           1       0.86      0.51      0.64      1104

    accuracy                           0.90      6507
   macro avg       0.89      0.75      0.79      6507
weighted avg       0.90      0.90      0.89      6507

offer


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6481
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      6507

aid_related
              precision    recall  f1-score   support

           0       0.79      0.85      0.82      3793
           1       0.77      0.69      0.73      2714

    accuracy                           0.79      6507
   macro avg       0.78      0.77      0.77      6507
weighted avg       0.78      0.79      0.78      6507

medical_help
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      5965
           1       0.67      0.09      0.17       542

    accuracy                           0.92      6507
   macro avg       0.80      0.54      0.56      6507
weighted avg       0.90      0.92      0.89      

count    9.000000
mean     0.859255
std      0.055942
min      0.774758
25%      0.821825
50%      0.887077
75%      0.892294
max      0.933391

In [4]:
#pipeline model 3
def pipeline_model_3():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline

In [5]:
model_3 = pipeline_model_3()

In [9]:
model_3.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline

In [10]:
parameters = {'features__vect__min_df': [1, 5],
              'features__tfidf__use_idf':[True, False],
          'clf__estimator__learning_rate':[0.01, 0.1, 0.5],
          'clf__estimator__n_estimators':[100, 200]}

In [11]:
model_cv_3 = GridSearchCV(estimator=model_3, 
                          scoring='f1_weighted', 
                          param_grid=parameters, 
                          verbose=3)

In [59]:
Y.sum()

related                   19906
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire                        282
earthqua

In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [212]:
%%time
model_3.fit(X_train, Y_train)

Wall time: 1min 42s


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [81]:
%%time
#make predictions
y_preds_3 = model_3.predict(X_test)

Wall time: 7.93 s


In [205]:
wavg_df_3 = metrics_report(Y_test, y_preds_3)

In [206]:
wavg_df_3

,precision,recall,f1-score,support
related,0.783542,0.801137,0.777111,6507.0
request,0.889641,0.896419,0.889729,6507.0
offer,0.992023,0.995697,0.993857,6507.0
aid_related,0.758949,0.759029,0.754306,6507.0
medical_help,0.912669,0.926848,0.912353,6507.0
medical_products,0.949186,0.956508,0.950013,6507.0
search_and_rescue,0.967028,0.973413,0.967423,6507.0
security,0.967202,0.978638,0.970599,6507.0
military,0.962603,0.968803,0.964185,6507.0
water,0.961374,0.963578,0.962092,6507.0


In [207]:
weighted_avg_report(wavg_df_3)

count    35.000000
mean      0.940781
std       0.057800
min       0.754306
25%       0.933265
50%       0.962092
75%       0.979511
max       0.993857
Name: f1-score, dtype: float64

'lowest quantile of f scores'

,precision,recall,f1-score,support
related,0.783542,0.801137,0.777111,6507.0
request,0.889641,0.896419,0.889729,6507.0
aid_related,0.758949,0.759029,0.754306,6507.0
medical_help,0.912669,0.926848,0.912353,6507.0
other_aid,0.841107,0.872599,0.845232,6507.0
infrastructure_related,0.907995,0.931458,0.914852,6507.0
weather_related,0.880764,0.881820,0.876986,6507.0
storm,0.931598,0.937606,0.931841,6507.0
direct_report,0.855848,0.865222,0.854408,6507.0


'highest quantile of f scores'

,precision,recall,f1-score,support
offer,0.992023,0.995697,0.993857,6507.0
clothing,0.986114,0.987859,0.986604,6507.0
missing_people,0.986705,0.989857,0.987283,6507.0
tools,0.987737,0.992777,0.990250,6507.0
hospitals,0.981502,0.987245,0.983908,6507.0
shops,0.991215,0.994467,0.992145,6507.0
aid_centers,0.980675,0.986937,0.983381,6507.0
fire,0.986286,0.989242,0.987045,6507.0
cold,0.979498,0.982634,0.979841,6507.0


In [180]:
metrics_report(Y_test, y_preds_3)

,precision,recall,f1-score,support
related,0.783542,0.801137,0.777111,6507.0
request,0.889641,0.896419,0.889729,6507.0
offer,0.992023,0.995697,0.993857,6507.0
aid_related,0.758949,0.759029,0.754306,6507.0
medical_help,0.912669,0.926848,0.912353,6507.0
medical_products,0.949186,0.956508,0.950013,6507.0
search_and_rescue,0.967028,0.973413,0.967423,6507.0
security,0.967202,0.978638,0.970599,6507.0
military,0.962603,0.968803,0.964185,6507.0
water,0.961374,0.963578,0.962092,6507.0


---
### Model 3 Run again, Tokenize has no stemming

End result mean of f-score is 0.94, slightly less than when stemming is present. Therefore decided to keep stemming in the tokenize function.

In [213]:
%%time
#make predictions
y_preds_3 = model_3.predict(X_test)

Wall time: 7.76 s


In [214]:
wavg_df_3 = metrics_report(Y_test, y_preds_3)

In [215]:
wavg_df_3

,precision,recall,f1-score,support
related,0.783542,0.801137,0.777111,6507.0
request,0.889641,0.896419,0.889729,6507.0
offer,0.992023,0.995697,0.993857,6507.0
aid_related,0.758949,0.759029,0.754306,6507.0
medical_help,0.912669,0.926848,0.912353,6507.0
medical_products,0.949186,0.956508,0.950013,6507.0
search_and_rescue,0.967028,0.973413,0.967423,6507.0
security,0.967202,0.978638,0.970599,6507.0
military,0.962603,0.968803,0.964185,6507.0
water,0.961374,0.963578,0.962092,6507.0


In [216]:
weighted_avg_report(wavg_df_3)

count    35.000000
mean      0.940781
std       0.057800
min       0.754306
25%       0.933265
50%       0.962092
75%       0.979511
max       0.993857
Name: f1-score, dtype: float64

'lowest quantile of f scores'

,precision,recall,f1-score,support
related,0.783542,0.801137,0.777111,6507.0
request,0.889641,0.896419,0.889729,6507.0
aid_related,0.758949,0.759029,0.754306,6507.0
medical_help,0.912669,0.926848,0.912353,6507.0
other_aid,0.841107,0.872599,0.845232,6507.0
infrastructure_related,0.907995,0.931458,0.914852,6507.0
weather_related,0.880764,0.881820,0.876986,6507.0
storm,0.931598,0.937606,0.931841,6507.0
direct_report,0.855848,0.865222,0.854408,6507.0


'highest quantile of f scores'

,precision,recall,f1-score,support
offer,0.992023,0.995697,0.993857,6507.0
clothing,0.986114,0.987859,0.986604,6507.0
missing_people,0.986705,0.989857,0.987283,6507.0
tools,0.987737,0.992777,0.990250,6507.0
hospitals,0.981502,0.987245,0.983908,6507.0
shops,0.991215,0.994467,0.992145,6507.0
aid_centers,0.980675,0.986937,0.983381,6507.0
fire,0.986286,0.989242,0.987045,6507.0
cold,0.979498,0.982634,0.979841,6507.0


In [217]:
metrics_report(Y_test, y_preds_3)

,precision,recall,f1-score,support
related,0.783542,0.801137,0.777111,6507.0
request,0.889641,0.896419,0.889729,6507.0
offer,0.992023,0.995697,0.993857,6507.0
aid_related,0.758949,0.759029,0.754306,6507.0
medical_help,0.912669,0.926848,0.912353,6507.0
medical_products,0.949186,0.956508,0.950013,6507.0
search_and_rescue,0.967028,0.973413,0.967423,6507.0
security,0.967202,0.978638,0.970599,6507.0
military,0.962603,0.968803,0.964185,6507.0
water,0.961374,0.963578,0.962092,6507.0


In [183]:
from sklearn.metrics import multilabel_confusion_matrix as confusion_matrix

In [196]:
def display_mat(Y_test, y_preds):
    '''
    prints a confusion matrix of each var in each label
    '''
    #accuracy_df = pd.DataFrame(columns=['Accuracy score'])
    actual = np.array(Y_test)
    col_names = list(Y_test.columns.values)
    for i in range(len(Y_test.columns)):
        key = col_names[i]
        confusion_mat = confusion_matrix(actual[:,i], y_preds[:,i])
        print(key, confusion_mat)
    #for pred, actual, col in zip(y_preds.transpose(), Y_test.values.transpose(), Y_test.columns):
        #results_dict[col] = accuracy_score(np.(Y_test[col]), y_preds[col])
    #return accuracy_df

In [197]:
display_mat(Y_test, y_preds_3)

related [[[4664  274]
  [1020  549]]

 [[ 549 1020]
  [ 274 4664]]]
request [[[ 618  486]
  [ 188 5215]]

 [[5215  188]
  [ 486  618]]]
offer [[[   0   26]
  [   2 6479]]

 [[6479    2]
  [  26    0]]]
aid_related [[[1682 1032]
  [ 536 3257]]

 [[3257  536]
  [1032 1682]]]
medical_help [[[ 143  399]
  [  77 5888]]

 [[5888   77]
  [ 399  143]]]
medical_products [[[ 123  224]
  [  59 6101]]

 [[6101   59]
  [ 224  123]]]
search_and_rescue [[[  45  147]
  [  26 6289]]

 [[6289   26]
  [ 147   45]]]
security [[[   6  127]
  [  12 6362]]

 [[6362   12]
  [ 127    6]]]
military [[[  71  154]
  [  49 6233]]

 [[6233   49]
  [ 154   71]]]
water [[[ 277  153]
  [  84 5993]]

 [[5993   84]
  [ 153  277]]]
food [[[ 514  217]
  [ 126 5650]]

 [[5650  126]
  [ 217  514]]]
shelter [[[ 329  269]
  [  93 5816]]

 [[5816   93]
  [ 269  329]]]
clothing [[[  44   57]
  [  22 6384]]

 [[6384   22]
  [  57   44]]]
money [[[  58  110]
  [  38 6301]]

 [[6301   38]
  [ 110   58]]]
missing_people [[[  12   5

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [261]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    '''
    Returns True/False boolean value for whether the first word in a string of text is a verb or not
    '''
    def starting_verb(self, text):
        pos_tags_dict = {}
        for ea_msg in nltk.sent_tokenize(msg):
            pos_tags_dict = dict(nltk.pos_tag(tokenize(ea_msg)))
            first_word = list(pos_tags_dict.keys())[0]
            first_tag = pos_tags_dict[first_word]
        #sentence_list = nltk.sent_tokenize(text)
        #for sentence in sentence_list:
            #pos_tags = nltk.pos_tag(tokenize(sentence))
            #first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [262]:
#pipeline model 4
def pipeline_model_4():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor()),
            #("word_count", WordCount()),
            #("character_count", CharacterCount()),
            #("noun_count", NounCount()),
            #("verb_count", VerbCount())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    #pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
    #                      ('tfidf', TfidfTransformer()),
    #                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])
    return pipeline

In [263]:
model_4 = pipeline_model_4()

In [264]:
%%time
y_preds_4 = model_4.fit(X_train, Y_train)

In [272]:
%%time
y_preds_4 = model_4.predict(X_test)#make predictions
wavg_df = metrics_report(Y_test, y_preds_4)
display(weighted_avg_report(wavg_df))

count    35.000000
mean      0.940778
std       0.057797
min       0.754306
25%       0.933265
50%       0.962092
75%       0.979511
max       0.993857
Name: f1-score, dtype: float64

'lowest quantile of f scores'

,precision,recall,f1-score,support
related,0.783542,0.801137,0.777111,6507.0
request,0.889641,0.896419,0.889729,6507.0
aid_related,0.758949,0.759029,0.754306,6507.0
medical_help,0.912669,0.926848,0.912353,6507.0
other_aid,0.841107,0.872599,0.845232,6507.0
infrastructure_related,0.907995,0.931458,0.914852,6507.0
weather_related,0.880764,0.881820,0.876986,6507.0
storm,0.931598,0.937606,0.931841,6507.0
direct_report,0.855848,0.865222,0.854408,6507.0


'highest quantile of f scores'

,precision,recall,f1-score,support
offer,0.992023,0.995697,0.993857,6507.0
clothing,0.986114,0.987859,0.986604,6507.0
missing_people,0.986705,0.989857,0.987283,6507.0
tools,0.987737,0.992777,0.990250,6507.0
hospitals,0.981502,0.987245,0.983908,6507.0
shops,0.991215,0.994467,0.992145,6507.0
aid_centers,0.980675,0.986937,0.983381,6507.0
fire,0.986094,0.989089,0.986934,6507.0
cold,0.979498,0.982634,0.979841,6507.0


None

Wall time: 20.2 s


In [273]:
def run_model(pipeline, X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y,random_state = 42)
    model = pipeline()
    y_preds = model.fit(X_train, Y_train)
    y_preds = model.predict(X_test)#make predictions
    wavg_df = metrics_report(Y_test, y_preds)
    display(weighted_avg_report(wavg_df))
    return model

### TRY RUNNING A PIPELINE MODEL WITH OBTAINED PARAMETERS

In [266]:
#pipeline model 5
def pipeline_model_5():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, 
                                         min_df=10
                                        )),
                ('tfidf', TfidfTransformer(use_idf=True))
            ])),

            ('starting_verb', StartingVerbExtractor()),
        ])),

        #('clf', MultiOutputClassifier(AdaBoostClassifier()))
        ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split=2, 
                                                             random_state=42, 
                                                             n_estimators=100))),
    ])
    return pipeline

In [274]:
%%time
model_5 = run_model(pipeline_model_5, X, Y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


count    35.000000
mean      0.936367
std       0.056137
min       0.773738
25%       0.929425
50%       0.952569
75%       0.973235
max       0.994010
Name: f1-score, dtype: float64

'lowest quantile of f scores'

,precision,recall,f1-score,support
related,0.794633,0.805748,0.773738,6507.0
request,0.897481,0.900722,0.889432,6507.0
aid_related,0.783533,0.784540,0.782462,6507.0
medical_help,0.905547,0.921469,0.893446,6507.0
other_aid,0.855229,0.875519,0.825356,6507.0
infrastructure_related,0.875306,0.934686,0.904022,6507.0
weather_related,0.885076,0.887198,0.885224,6507.0
other_weather,0.941519,0.950361,0.928044,6507.0
direct_report,0.865105,0.866144,0.845579,6507.0


'highest quantile of f scores'

,precision,recall,f1-score,support
offer,0.992025,0.996004,0.994010,6507.0
clothing,0.983399,0.985861,0.980667,6507.0
missing_people,0.989809,0.989703,0.984731,6507.0
electricity,0.979875,0.981558,0.973642,6507.0
tools,0.987743,0.993853,0.990789,6507.0
hospitals,0.978296,0.989089,0.983663,6507.0
shops,0.989271,0.994621,0.991939,6507.0
aid_centers,0.977992,0.988935,0.983433,6507.0
fire,0.989358,0.989242,0.984183,6507.0


None

Wall time: 5min 34s


In [ ]:
AdaBoostClassifier

In [281]:
#pipeline model 6
def pipeline_model_6():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, 
                                         min_df=10
                                        )),
                ('tfidf', TfidfTransformer(use_idf=True))
            ])),

            ('starting_verb', StartingVerbExtractor()),
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
        #('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split=2, 
                                                             #random_state=42, 
                                                             #n_estimators=100))),
    ])
    return pipeline

In [282]:
%%time
model_6 = run_model(pipeline_model_6, X, Y)

count    35.000000
mean      0.940143
std       0.058305
min       0.751381
25%       0.928964
50%       0.960996
75%       0.978966
max       0.993703
Name: f1-score, dtype: float64

'lowest quantile of f scores'

,precision,recall,f1-score,support
related,0.780339,0.798832,0.774573,6507.0
request,0.887983,0.895036,0.887900,6507.0
aid_related,0.756138,0.756262,0.751381,6507.0
medical_help,0.912740,0.926848,0.912871,6507.0
other_aid,0.843370,0.873982,0.846369,6507.0
infrastructure_related,0.906537,0.931919,0.913467,6507.0
weather_related,0.880345,0.881820,0.877364,6507.0
storm,0.924548,0.932073,0.922696,6507.0
direct_report,0.856269,0.865529,0.854233,6507.0


'highest quantile of f scores'

,precision,recall,f1-score,support
offer,0.992022,0.995390,0.993703,6507.0
clothing,0.986024,0.988013,0.986267,6507.0
missing_people,0.987621,0.990318,0.987433,6507.0
tools,0.988915,0.993238,0.990748,6507.0
hospitals,0.983352,0.988320,0.984970,6507.0
shops,0.989267,0.993853,0.991555,6507.0
aid_centers,0.981683,0.986630,0.983806,6507.0
fire,0.987132,0.989857,0.987179,6507.0
cold,0.979290,0.982480,0.979717,6507.0


None

Wall time: 2min 30s


In [288]:
model_6.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents

In [ ]:
#grid search

parameters_3 = {'vect__min_df': [5, 10],
              'clf__estimator__n_estimators':[100, 500, 1000], 
              'clf__estimator__min_samples_split':[1, 2]}

In [ ]:
model_cv_3 = GridSearchCV(estimator=model_3, param_grid=parameters_3, verbose=3)

In [ ]:
%%time
model_cv_3.fit(X_train, Y_train)

model CV 1 best parameters:

     {'clf__estimator__min_samples_split': 2,
     'clf__estimator__n_estimators': 100,
     'tfidf__use_idf': True,
     'vect__min_df': 5}

model CV 2 best parameters:

    {'clf__estimator__min_samples_split': 2,
     'clf__estimator__n_estimators': 1000,
     'tfidf__use_idf': True,
     'vect__binary': False,
     'vect__min_df': 10}

F-scores

    Model 1:
    count    35.000000
    mean      0.931140
    std       0.057876
    min       0.771124
    25%       0.916096
    50%       0.944198
    75%       0.971679
    max       0.994010

    Model CV 1:
    count    9.000000
    mean     0.851604
    std      0.053219
    min      0.771124
    25%      0.817647
    50%      0.872271
    75%      0.883507
    max      0.917431

    Model 2:
    count    35.000000
    mean      0.930772
    std       0.058746
    min       0.767192
    25%       0.921509
    50%       0.940264
    75%       0.970957
    max       0.994010

    Model CV 2:
    count    9.000000
    mean     0.859255
    std      0.055942
    min      0.774758
    25%      0.821825
    50%      0.887077
    75%      0.892294
    max      0.933391

    Model 3:
    count    35.000000
    mean      0.940781
    std       0.057800
    min       0.754306
    25%       0.933265
    50%       0.962092
    75%       0.979511
    max       0.993857

    Model 4:
    count    35.000000
    mean      0.940778
    std       0.057797
    min       0.754306
    25%       0.933265
    50%       0.962092
    75%       0.979511
    max       0.993857

    Model 5:
    count    35.000000
    mean      0.936367
    std       0.056137
    min       0.773738
    25%       0.929425
    50%       0.952569
    75%       0.973235
    max       0.994010

    Model 6:
    count    35.000000
    mean      0.940143
    std       0.058305
    min       0.751381
    25%       0.928964
    50%       0.960996
    75%       0.978966
    max       0.993703

### 9. Export your model as a pickle file

In [22]:
def pickle_file(file_to_pickle, file_name):
    '''
    Saves a file to the data folder with the extension .pkl
    file path: '../data/'+ file_name+'.pkl'
    '''
    pickle.dump(file_to_pickle, open('../data/'+file_name+'.pkl', 'wb'))

In [41]:
%%time
pickle_file(model_3, '../data/model_3')

Wall time: 106 ms


In [434]:
%%time
pickle_file(model_cv_2, 'model_cv_2.pkl')

Wall time: 3min 34s


In [439]:
model_cv_load = load_pickle('model_cv.pkl')

AttributeError: 'str' object has no attribute 'pop'

Note: the cross-validation models (which are too big for GitHub anyways) were not able to be saved at the time.  I encountered a bug where the tokenize instance doesn't match the named function, possibly because the function was editing while the tests were running. The summary results are saved as outputs in this notebook.

In [275]:
pickle_file(model_4, 'model_4')

In [276]:
pickle_file(model_5, 'model_5')

In [290]:
pickle_file(model_6, 'model_6')

In [86]:
def load_pickle(pickled_file_name):
    '''
    Loads a pickled file (.pkl) in the data folder. 
    File path: 
    '''
    return pickle.load(open('../data/'+ pickled_file_name+'.pkl', 'rb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.